<a href="https://colab.research.google.com/github/NestorSaenz/Desafios-1-y-2/blob/main/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparación de entorno



In [ ]:
!pip install sentence-transformers
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install ocrmypdf PyPDF2
!apt-get install -y ghostscript
!pip install flair
!pip install transformers


#Librerías

In [ ]:
import re
from google.colab import files
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import pipeline
from flair.data import Sentence
from flair.models import SequenceTagger
import ocrmypdf
import PyPDF2

#Carga de archivos pdf (CVs)

In [4]:
# Configura las carpetas donde se guardarán los PDFs y los textos extraídos
carpeta_pdf = './CVs'
carpeta_txt = './Texto_2'

# Crear las carpetas si no existen
os.makedirs(carpeta_pdf, exist_ok=True)
os.makedirs(carpeta_txt, exist_ok=True)

# Cargar archivos desde la máquina local
uploaded = files.upload()

# Guardar los archivos cargados en la carpeta designada
for archivo in uploaded.keys():
    # Mover los archivos cargados a la carpeta de PDFs
    os.rename(archivo, os.path.join(carpeta_pdf, archivo))

print(f"Archivos cargados: {list(uploaded.keys())}")


Saving 1724342646_CV1.pdf to 1724342646_CV1.pdf
Saving 1724342661_CV2.pdf to 1724342661_CV2.pdf
Saving 1724342673_CV3.pdf to 1724342673_CV3.pdf
Saving 1724342683_CV4.pdf to 1724342683_CV4.pdf
Saving 1724342694_CV5.pdf to 1724342694_CV5.pdf
Saving 1724342704_CV6.pdf to 1724342704_CV6.pdf
Saving 1724342714_CV7.pdf to 1724342714_CV7.pdf
Saving 1724342728_CV8.pdf to 1724342728_CV8.pdf
Saving 1724342738_CV9.pdf to 1724342738_CV9.pdf
Archivos cargados: ['1724342646_CV1.pdf', '1724342661_CV2.pdf', '1724342673_CV3.pdf', '1724342683_CV4.pdf', '1724342694_CV5.pdf', '1724342704_CV6.pdf', '1724342714_CV7.pdf', '1724342728_CV8.pdf', '1724342738_CV9.pdf']


#Procesamiento de archivos pdf

In [ ]:
# Paso 2: Configuración de carpetas
carpeta_pdf = '/content/CVs'
carpeta_txt = '/content/Texto_2'

os.makedirs(carpeta_pdf, exist_ok=True)
os.makedirs(carpeta_txt, exist_ok=True)


# Paso 4: Procesar los PDFs
for archivo in os.listdir(carpeta_pdf):
    if archivo.endswith('.pdf'):
        archivo_pdf = os.path.join(carpeta_pdf, archivo)

        archivo_pdf_con_ocr = os.path.join(carpeta_txt, f'{os.path.splitext(archivo)[0]}_ocr.pdf')

        # Aplicar OCR al PDF
        ocrmypdf.ocr(archivo_pdf, archivo_pdf_con_ocr, force_ocr=True)# reconocimiento optico de caracteres

        # Extraer texto del PDF procesado
        texto_extraido = ""
        with open(archivo_pdf_con_ocr, 'rb') as pdf_con_ocr:
            pdf_reader = PyPDF2.PdfReader(pdf_con_ocr)
            for pagina in pdf_reader.pages:
                texto_extraido += pagina.extract_text()

        # Guardar texto extraído como archivo .txt
        archivo_salida = os.path.join(carpeta_txt, f'{os.path.splitext(archivo)[0]}.txt')
        with open(archivo_salida, 'w', encoding='utf-8') as f:
            f.write(texto_extraido)

        print(f'Texto extraído y guardado de: {archivo}')

#Desarrollo y pruebas

##Funciones

In [6]:
def normalizar_texto(texto):
    """
    Normaliza el texto eliminando caracteres invisibles, espacios extra y puntuación innecesaria.

    Args:
        texto (str): Texto a normalizar.

    Returns:
        str: Texto normalizado.
    """
    # Eliminar caracteres no alfabéticos o espaciales y convertir a minúsculas
    texto_normalizado = re.sub(r'[^\w\s]', '', texto.lower())  # Elimina puntuación
    texto_normalizado = re.sub(r'\s+', ' ', texto_normalizado).strip()  # Elimina espacios extra
    return texto_normalizado

In [7]:
def calcular_coincidencias(cv, frases_ia):
    """
    Calcula las coincidencias exactas de palabras o frases en un CV y devuelve las frases coincidentes.

    Args:
        cv (str): Texto del CV.
        frases_ia (list): Lista de palabras o frases clave.

    Returns:
        tuple: Número de coincidencias y lista de frases coincidentes.
    """
    # Normalizamos el CV
    cv_normalizado = normalizar_texto(cv)

    frases_encontradas = []
    for frase in frases_ia:
        frase_normalizada = normalizar_texto(frase)  # Normalizamos la frase clave

        # Verificamos si la frase normalizada está contenida en el CV normalizado
        if frase_normalizada in cv_normalizado:
            frases_encontradas.append(frase)

    return len(frases_encontradas), frases_encontradas

In [8]:
def mostrar_similitudes_semanticas(cvs:list, frases_ia:list, similitudes:np.array, limite_caracteres:int=50) -> None:
    """
    Muestra los primeros caracteres de cada CV y sus similitudes semánticas promedio con frases IA.

    Args:
        cvs (list): Lista de textos de los CVs.
        frases_ia (list): Lista de frases relacionadas con IA.
        similitudes (ndarray): Matriz de similitudes calculadas (CVs x frases IA).
        limite_caracteres (int): Número de caracteres iniciales a mostrar por CV.
    """
    print("Similitudes semánticas por CV:")
    for i, cv in enumerate(cvs):
        texto_corto = cv[:limite_caracteres] + "..." if len(cv) > limite_caracteres else cv
        similitud_promedio = similitudes[i].mean()
        print(f"CV {i+1}:")
        print(f"Texto: {texto_corto}")
        print(f"Similitud semántica promedio: {similitud_promedio:.4f}\n")

In [9]:
def calcular_puntaje(cvs:list, frases_ia:list, embeddings_cvs:list, embeddings_frases:list, w1:float=0.8, w2:float=0.2, factor_coincidencias:int=16, factor_similitud:int=9) -> list:
    """
    Calcula el puntaje ajustado para cada CV y lo escala individualmente entre 1 y 10,
    permitiendo ajustar los pesos de las coincidencias exactas y las similitudes semánticas,
    con factores adicionales para amplificar las influencias.

    Args:
        cvs (list): Lista de textos de los CVs.
        frases_ia (list): Lista de palabras y frases relacionadas con IA.
        embeddings_cvs (list): Embeddings de los CVs.
        embeddings_frases (list): Embeddings de las frases/ palabras clave de IA.
        w1 (float): Peso para las coincidencias exactas (entre 0 y 1).
        w2 (float): Peso para las similitudes semánticas (entre 0 y 1).
        factor_coincidencias (float): Factor para amplificar el impacto de las coincidencias exactas.
        factor_similitud (float): Factor para amplificar el impacto de la similitud semántica.

    Returns:
        list: Puntajes escalados entre 1 y 10 para cada CV.
    """
    puntajes_ajustados = []

    # Asegurarse de que la suma de w1 y w2 sea 1
    if w1 + w2 != 1:
        raise ValueError("La suma de los pesos w1 y w2 debe ser 1.")

    # Para cada CV
    for cv, embedding_cv in zip(cvs, embeddings_cvs):
        coincidencias = 0
        similitudes_semanticas = []

        # Calcular coincidencias exactas
        for frase in frases_ia:
            if frase.lower() in cv.lower():
                coincidencias += 1

        # Calcular similitudes semánticas
        for embedding_frase in embeddings_frases_ia:
            similitud = np.dot(embedding_cv, embedding_frase) / (np.linalg.norm(embedding_cv) * np.linalg.norm(embedding_frase))
            similitudes_semanticas.append(similitud)

        # Promedio de similitudes semánticas
        similitud_promedio = np.mean(similitudes_semanticas) if similitudes_semanticas else 0

        # Amplificar el impacto de las coincidencias exactas y similitudes semánticas
        coincidencias_ajustadas = coincidencias * factor_coincidencias
        similitud_ajustada = similitud_promedio * factor_similitud

        # Calcular el puntaje ajustado
        puntaje_ajustado = (w1 * coincidencias_ajustadas) + (w2 * similitud_ajustada)

        # Escalar el puntaje entre 1 y 10
        min_local = 0  # valor mínimo esperado (puede ajustarse)
        max_local = 30  # valor máximo esperado (puede ajustarse)

        puntaje_escalado = 1 + 9 * (puntaje_ajustado - min_local) / (max_local - min_local)

        # Agregar el puntaje escalado a la lista
        puntajes_ajustados.append(puntaje_escalado)

    return puntajes_ajustados


##Generación de embeddings

###Resumen del código

El objetivo es analizar CVs en formato texto para determinar su relevancia hacia temas relacionados con IA. Utiliza dos enfoques principales:

1. **Coincidencias exactas:** Identifica frases clave relacionadas con IA dentro de los textos de los CVs.
2. **Similitud semántica:** Calcula qué tan similares son los textos de los CVs a una lista de frases clave, evaluando el significado.

###Componentes clave

####**Modelo de lenguaje preentrenado**
Se utiliza un modelo `SentenceTransformer` preentrenado (`distilbert-base-nli-stsb-mean-tokens`) para convertir tanto las frases clave como los CVs en vectores numéricos (embeddings). Esto permite medir similitudes semánticas entre los textos.

####**Lista de frases clave**
Se define una lista extensa de términos y frases relacionadas con IA, como "Machine Learning", "Deep Learning", "Redes Neuronales", etc. Estas frases son el punto de referencia para evaluar la relevancia de los CVs.

####**Carga de CVs**
Los archivos .txt que contienen los CVs se leen desde una carpeta específica y su contenido se almacena en una lista para procesarlos posteriormente.

###Procesos principales

####**Cálculo de similitud semántica**
Se generan embeddings para los CVs y las frases clave. Luego, se utiliza la similitud de coseno para medir qué tan cercanos son los embeddings de los CVs a los de las frases clave, permitiendo una evaluación semántica más profunda.

####**Búsqueda de coincidencias exactas**
Se normaliza el texto de los CVs y las frases clave (eliminando puntuación y caracteres invisibles). Se busca si las frases clave aparecen exactamente en cada CV, devolviendo la cantidad de coincidencias y las frases encontradas.

###Resultados esperados

- **Coincidencias exactas:** Se informa cuántas frases clave aparecen en cada CV y cuáles son.
- **Similitudes semánticas:** Se calcula y muestra el promedio de similitudes para cada CV, lo que permite evaluar la relación general del texto con los temas de IA.

###Aplicaciones
Este análisis puede ser útil para filtrar y priorizar CVs en procesos de selección relacionados con puestos en IA, ayudando a identificar candidatos con experiencia o interés en el área.


In [10]:
# Cargar el modelo preentrenado para generar los embeddings
modelo = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Lista de frases clave relacionadas con IA
frases_ia = [
    "Machine Learning", "Deep Learning", "Artificial Intelligence", "Neural Networks",
    "Computer Vision", "Natural Language Processing", "Reinforcement Learning",
    "Supervised Learning", "Unsupervised Learning", "Semi-Supervised Learning",
    "Transfer Learning", "Explainable AI", "Edge Computing", "Generative AI",
    "Big Data", "Data Mining", "Data Science", "Feature Engineering", "Model Optimization",
    "Predictive Analytics", "Clustering Algorithms", "Classification Algorithms",
    "Regression Algorithms", "Decision Trees", "Support Vector Machines",
    "Convolutional Neural Networks", "Recurrent Neural Networks", "Transformers",
    "Gradient Boosting", "Hyperparameter Tuning", "AutoML", "AI Ethics", "Fairness in AI",
    "Bias Detection", "GPT", "BERT", "LLM (Large Language Models)", "NLP", "OCR",
    "Speech Recognition", "Text-to-Speech", "Speech-to-Text", "GANs (Generative Adversarial Networks)",
    "RL (Reinforcement Learning)", "AI for Healthcare", "AI for Finance", "AI for Education",
    "AI for Industry 4.0", "Autonomous Systems", "AI Deployment", "AI Scalability", "AI Frameworks",
    "AI Research", "Inteligencia Artificial", "Aprendizaje Automático", "Aprendizaje Supervisado",
    "Aprendizaje No Supervisado", "Aprendizaje Semi-Supervisado", "Aprendizaje por Refuerzo",
    "Aprendizaje Profundo", "Redes Neuronales", "Visión por Computadora",
    "Procesamiento de Lenguaje Natural", "Analítica Predictiva", "Minería de Datos", "Ciencia de Datos",
    "Ingeniería de Características", "Optimización de Modelos", "Análisis Predictivo", "Clústeres",
    "Algoritmos de Clasificación", "Algoritmos de Regresión", "Árboles de Decisión",
    "Máquinas de Soporte Vectorial", "Redes Neuronales Convolucionales",
    "Redes Neuronales Recurrentes", "Transformadores", "Ajuste de Hiperparámetros",
    "Modelos Explicables", "Modelos de Lenguaje Grande", "Toma de Decisiones Autónomas",
    "Inteligencia Artificial Responsable", "Ética en IA", "Detección de Sesgos", "Reconocimiento de Voz",
    "Texto a Voz", "Voz a Texto", "Generación de Imágenes", "Sistemas Autónomos", "IA en la Salud",
    "IA en Finanzas", "IA en la Industria", "IA en la Educación", "Computación en el Borde",
    "Datos Masivos", "Automatización de Procesos", "Investigación en IA"
]


In [12]:
# Generar embeddings para las frases clave
embeddings_frases_ia = modelo.encode(frases_ia)

In [ ]:
# Mostrar algunos embeddings generados
print(f"Ejemplo de embedding para la frase 'Machine Learning':\n{embeddings_frases_ia[0]}")

In [14]:
len(embeddings_frases_ia[0])

768

In [15]:
# Carpeta donde están los archivos .txt generados
carpeta_txt = './Texto_2'

# Lista para almacenar el contenido de los CVs
cvs = []

# Leer cada archivo .txt en la carpeta
for archivo in os.listdir(carpeta_txt):
    if archivo.endswith('.txt'):  # Solo procesar archivos .txt
        archivo_ruta = os.path.join(carpeta_txt, archivo)
        with open(archivo_ruta, 'r', encoding='utf-8') as f:
            contenido = f.read()
            cvs.append(contenido)  # Agregar el contenido del archivo a la lista

# Verificar que se hayan cargado los textos correctamente
print(f"Se cargaron {len(cvs)} CVs en la lista.")


Se cargaron 9 CVs en la lista.


In [ ]:
cvs

In [17]:
# Generar embeddings para los CVs
embeddings_cvs = modelo.encode(cvs)

In [ ]:
# Mostrar ejemplo de un embedding generado
print(f"Ejemplo de embedding para el CV 1:\n{embeddings_cvs[0]}")

In [19]:
len(embeddings_cvs[0])

768

In [20]:
# Calcular la similitud de coseno entre los embeddings de los CVs y las frases clave
similitudes = cosine_similarity(embeddings_cvs, embeddings_frases_ia)

In [22]:
# Mostrar coincidencias exactas para cada CV
for i, cv in enumerate(cvs):
    num_coincidencias, coincidencias = calcular_coincidencias(cv, frases_ia)
    print(f"CV {i+1}:")
    print(f"Texto: {cv[:50]}...")  # Muestra los primeros 50 caracteres del CV para contexto
    print(f"Coincidencias exactas: {num_coincidencias}")

    if num_coincidencias > 0:
        print(f"Frases coincidentes: {', '.join(coincidencias)}")  # Muestra todas las frases coincidentes
    else:
        print("No se encontraron coincidencias exactas.")

    print("\n")

CV 1:
Texto: Michael  Smith
BI / Big Data/ Azure
Manchester,  U...
Coincidencias exactas: 1
Frases coincidentes: Big Data


CV 2:
Texto: POWELL
FINWOOD
WORK  EXPERIENCE
Account  Sales Exe...
Coincidencias exactas: 0
No se encontraron coincidencias exactas.


CV 3:
Texto: Phone  and WhatsApp:
+62 8577 7124 773
Email:
dhik...
Coincidencias exactas: 0
No se encontraron coincidencias exactas.


CV 4:
Texto: Dyah
Hediyati  $S.Kom
RINGKASAN  PROFESIONAL
Lulus...
Coincidencias exactas: 0
No se encontraron coincidencias exactas.


CV 5:
Texto: Alice Clark
Al / Machine  Learning
Delhi, India Em...
Coincidencias exactas: 3
Frases coincidentes: Machine Learning, Natural Language Processing, Big Data


CV 6:
Texto: MOH PUJI JUNAEDI
FULLSTACK DEVELOPER
Bojonegoro, J...
Coincidencias exactas: 1
Frases coincidentes: Machine Learning


CV 7:
Texto: DR.SANTOSH  KAKADE
a
DR.SANTOSH  KAKADE
MS (Surger...
Coincidencias exactas: 0
No se encontraron coincidencias exactas.


CV 8:
Texto: Education
Technical 

In [23]:
mostrar_similitudes_semanticas(cvs, frases_ia, similitudes)

Similitudes semánticas por CV:
CV 1:
Texto: Michael  Smith
BI / Big Data/ Azure
Manchester,  U...
Similitud semántica promedio: 0.1193

CV 2:
Texto: POWELL
FINWOOD
WORK  EXPERIENCE
Account  Sales Exe...
Similitud semántica promedio: 0.0266

CV 3:
Texto: Phone  and WhatsApp:
+62 8577 7124 773
Email:
dhik...
Similitud semántica promedio: 0.0331

CV 4:
Texto: Dyah
Hediyati  $S.Kom
RINGKASAN  PROFESIONAL
Lulus...
Similitud semántica promedio: 0.2087

CV 5:
Texto: Alice Clark
Al / Machine  Learning
Delhi, India Em...
Similitud semántica promedio: 0.1116

CV 6:
Texto: MOH PUJI JUNAEDI
FULLSTACK DEVELOPER
Bojonegoro, J...
Similitud semántica promedio: 0.1299

CV 7:
Texto: DR.SANTOSH  KAKADE
a
DR.SANTOSH  KAKADE
MS (Surger...
Similitud semántica promedio: 0.1212

CV 8:
Texto: Education
Technical  Skills
Professional
Experienc...
Similitud semántica promedio: 0.1518

CV 9:
Texto: Ringgi  Cahyo  Dwiputra
South Tangerang,  Banten  ...
Similitud semántica promedio: 0.1604



In [24]:
# Llamada a la función para calcular los puntajes ajustados y escalados
puntajes_escalados = calcular_puntaje(cvs, frases_ia, embeddings_cvs, embeddings_frases_ia)

# Mostrar los puntajes escalados con los primeros 50 caracteres de cada CV
for cv, puntaje_escalado in zip(cvs, puntajes_escalados):
    print(f"CV: {cv[:50]}... -> Puntaje escalado: {puntaje_escalado:.2f}\n\n")


CV: Michael  Smith
BI / Big Data/ Azure
Manchester,  U... -> Puntaje escalado: 4.90


CV: POWELL
FINWOOD
WORK  EXPERIENCE
Account  Sales Exe... -> Puntaje escalado: 1.01


CV: Phone  and WhatsApp:
+62 8577 7124 773
Email:
dhik... -> Puntaje escalado: 1.02


CV: Dyah
Hediyati  $S.Kom
RINGKASAN  PROFESIONAL
Lulus... -> Puntaje escalado: 1.11


CV: Alice Clark
Al / Machine  Learning
Delhi, India Em... -> Puntaje escalado: 4.90


CV: MOH PUJI JUNAEDI
FULLSTACK DEVELOPER
Bojonegoro, J... -> Puntaje escalado: 1.07


CV: DR.SANTOSH  KAKADE
a
DR.SANTOSH  KAKADE
MS (Surger... -> Puntaje escalado: 1.07


CV: Education
Technical  Skills
Professional
Experienc... -> Puntaje escalado: 1.08


CV: Ringgi  Cahyo  Dwiputra
South Tangerang,  Banten  ... -> Puntaje escalado: 1.09




##Extracción de datos

###Resumen del código

El objetivo es analizar textos de CVs para extraer nombres de personas, correos electrónicos y números de teléfono utilizando técnicas avanzadas de procesamiento de lenguaje natural (NLP). Se evalúan diferentes estrategias, se combinan modelos para obtener mejores resultados, y se descarta una alternativa basada en SpaCy debido a su bajo rendimiento.

###Modelos y herramientas utilizados

####**Transformers (Hugging Face)**
Se utilizó el modelo `dbmdz/bert-large-cased-finetuned-conll03-english` para el reconocimiento de entidades nombradas (NER), identificando entidades del tipo `PER` (personas).

####**Expresiones regulares**
Se emplearon patrones para detectar correos electrónicos y teléfonos en varios formatos.

####**SpaCy**
Aunque se probó como alternativa para el reconocimiento de nombres, no ofreció buenos resultados y se descartó.

###Procesos principales

####**Extracción con Transformers**
1. Se utiliza un pipeline NER de Hugging Face para identificar entidades en los CVs.
2. Se filtran las entidades etiquetadas como nombres de personas (`B-PER` e `I-PER`).
3. Se procesan los tokens divididos (subpalabras marcadas con `##`) para reconstruir nombres completos.

####**Búsqueda de correos y teléfonos**
- **Correos electrónicos:** Se emplea una expresión regular para detectar direcciones de email.
- **Números de teléfono:** Se diseñaron expresiones regulares avanzadas para capturar formatos internacionales y locales.

###Estrategias de mejora

####**Pruebas y ajustes**
1. **Primera prueba:** Extracción inicial con NER y expresiones regulares básicas. Se identificaron problemas con subpalabras y formatos de teléfonos.
2. **Segunda prueba:** Mejoras en las expresiones regulares para teléfonos y eliminación de subpalabras en los nombres.
3. **Tercera prueba:** Uso de una estrategia de "agregación simple" en el pipeline NER para unificar resultados y evitar duplicados.

####**Combinación de modelos**
Se combinaron resultados de diferentes implementaciones del modelo NER para maximizar la cantidad y precisión de los nombres detectados, ya que cada enfoque capturaba nombres que el otro omitía.

###Resultados esperados
1. **Nombres:** Lista completa y unificada de nombres detectados en cada CV.
2. **Correos electrónicos:** Identificación precisa de direcciones de email.
3. **Teléfonos:** Captura de números en formatos internacionales y locales.

###Aplicaciones
Este código puede ser utilizado en procesos de reclutamiento automatizado, analizando grandes volúmenes de CVs para extraer información clave de contacto y nombres, facilitando la organización y clasificación de candidatos.


In [25]:
# Usamos el pipeline de NER de Hugging Face
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def extract_names_and_emails_and_phones_with_transformers(cvs):
    # Lista para almacenar los resultados
    results = []

    # Expresiones regulares para buscar correos electrónicos y teléfonos
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zAZ0-9.-]+\.[a-zA-Z]{2,}'
    phone_pattern = r'\+?\(?\d{1,3}\)?[\s\-]?\(?\d{1,4}\)?[\s\-]?\d{1,4}[\s\-]?\d{1,4}'  # Teléfonos con varios formatos

    for cv in cvs:
        # Buscar correos electrónicos y teléfonos
        emails = re.findall(email_pattern, cv)
        phones = re.findall(phone_pattern, cv)

        # Obtener las entidades nombradas usando transformers
        entities = ner_pipeline(cv)

        # Filtrar por personas (entidades de tipo 'PER')
        names = [entity['word'] for entity in entities if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER']

        # Añadir a los resultados
        results.append({
            'Nombres': names,
            'Emails': emails,
            'Teléfonos': phones
        })

    return results

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [26]:
# Extraer nombres, correos y teléfonos
extracted_data = extract_names_and_emails_and_phones_with_transformers(cvs)

# Mostrar los resultados
for idx, data in enumerate(extracted_data, 1):
    print(f"CV {idx}:")
    print("Nombres:", data['Nombres'])
    print("Emails:", data['Emails'])
    print("Teléfonos:", data['Teléfonos'])
    print("-" * 40)

CV 1:
Nombres: ['Michael']
Emails: []
Teléfonos: ['140749', '2015', '2014', '2007']
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com', 'hello@reallygreatsite.com']
Teléfonos: ['2023', '2025', '2021', '2023', '2021', '2021', '+123-456-7890', '2021', '2019', '2023', '2021', '+123-456-7890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+62 8577 7124 773', '2002', '2017', '2018\n2019', '2019\n2017', '2020', '2020\n2020', '2021', '2021', '2022', '2021', '2021', '2021', '2021', '2008', '2014', '2014', '2017', '2017', '2020', '2020', '2021', '2017', '2019', '2019', '2021-12-22']
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['2021', '2014', '2015', '2017', '2019', '2019', '2019', '+62 85287404232', '2019', '2020', '2019', '2021', '2021', '2018', '2018', '2019']
----------------------------------------
CV 5:
Nombres: []
Emails: []

In [29]:
# Cargar el pipeline de NER de transformers
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Función para extraer nombres de personas usando NER
def extract_names_with_ner(text):
    # Ejecutar NER sobre el texto
    entities = ner(text)

    # Unir los tokens de nombres que están divididos en subpalabras
    names = []
    current_name = []

    for entity in entities:
        # Filtrar solo las entidades que son personas (label 'PER')
        if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
            word = entity['word']

            # Si el token tiene '##', es parte de una palabra anterior, así que lo añadimos
            if word.startswith('##'):
                if current_name:  # Asegurarnos de que current_name no esté vacío
                    current_name[-1] += word[2:]  # Añadir a la palabra anterior
            else:
                if current_name:  # Añadir el nombre completo anterior
                    names.append(" ".join(current_name))
                current_name = [word]  # Iniciar un nuevo nombre

    # Añadir el último nombre si lo hay
    if current_name:
        names.append(" ".join(current_name))

    return names

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
# Extraer la información de cada CV
for i, cv in enumerate(cvs, start=1):
    print(f"CV {i}:")

    # Extraer nombres usando transformers
    names = extract_names_with_ner(cv)
    print(f"Nombres: {names}")

    # Extraer emails usando NER (para ejemplo, aunque no es necesario con transformers)
    emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', cv)
    print(f"Emails: {emails}")

    # Extraer teléfonos con el mismo enfoque anterior
    phones = re.findall(r'(\+?\d{1,4}[-\s]?)?(\(?\d{1,4}\)?[-\s]?)?[\d]{6,13}', cv)
    print(f"Teléfonos: {[''.join(phone).strip() for phone in phones if len(''.join(phone).strip()) > 8]}")

    print("-" * 40)

CV 1:
Nombres: ['Michael']
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com', 'hello@reallygreatsite.com']
Teléfonos: []
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: []
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: []
----------------------------------------
CV 5:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: ['lam', 'Junaedi']
Emails: ['me@junae.id']
Teléfonos: []
----------------------------------------
CV 7:
Nombres: []
Emails: ['drsantoshkakade@gmail.com', 'drsantoshkakade@ciilm.com']
Teléfonos: []
----------------------------------------
CV 8:
Nombres: []
Emails: ['loren@shevitz.org']
Teléfonos: []
----------------------------------------
CV 9:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------


In [31]:
# Cargar el pipeline de NER con aggregation_strategy="simple"
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')

email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text):
    # Detectar entidades con el modelo
    entities = ner(text)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Procesar entidades detectadas
    for entity in entities:
        if entity['entity_group'] == 'PER':  # Nombres de personas
            names.add(entity['word'])

    return {
        "names": sorted(names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [32]:
# Procesar cada CV
for i, cv in enumerate(cvs, start=1):
    info = extract_info_with_ner(cv)
    print(f"CV {i}:")
    print(f"Nombres: {info['names']}")
    print(f"Emails: {info['emails']}")
    print(f"Teléfonos: {info['phones']}")
    print("-" * 40)

CV 1:
Nombres: ['Michael']
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com']
Teléfonos: ['+1234567890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+6285777124']
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['+6285287404232']
----------------------------------------
CV 5:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: ['lam Junaedi']
Emails: ['me@junae.id']
Teléfonos: ['+6282331472499']
----------------------------------------
CV 7:
Nombres: ['##v']
Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com']
Teléfonos: ['02024444490', '09422071490', '09850671175']
----------------------------------------
CV 8:
Nombres: []
Emails: ['loren@shevitz.org']
Teléfonos: ['606142853', '7736651234']
----------------------------------------
CV 9:
Nombres

In [33]:
# Cargar el pipeline de NER
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')

email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text):
    # Detectar entidades con el modelo
    entities = ner(text)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Procesar entidades detectadas
    for entity in entities:
        if entity['entity_group'] == 'PER':  # Nombres de personas
            # Eliminar fragmentos innecesarios como '##v' o tokens mal formateados
            if not entity['word'].startswith('##'):
                names.add(entity['word'])

    return {
        "names": sorted(names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [34]:
# Procesar cada CV
for i, cv in enumerate(cvs, start=1):
    info = extract_info_with_ner(cv)
    print(f"CV {i}:")
    print(f"Nombres: {info['names']}")
    print(f"Emails: {info['emails']}")
    print(f"Teléfonos: {info['phones']}")
    print("-" * 40)

CV 1:
Nombres: ['Michael']
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com']
Teléfonos: ['+1234567890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+6285777124']
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['+6285287404232']
----------------------------------------
CV 5:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: ['lam Junaedi']
Emails: ['me@junae.id']
Teléfonos: ['+6282331472499']
----------------------------------------
CV 7:
Nombres: []
Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com']
Teléfonos: ['02024444490', '09422071490', '09850671175']
----------------------------------------
CV 8:
Nombres: []
Emails: ['loren@shevitz.org']
Teléfonos: ['606142853', '7736651234']
----------------------------------------
CV 9:
Nombres: []


In [35]:
# Cargar el pipeline de NER
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')

email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text):
    # Detectar entidades con el modelo
    entities = ner(text)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Procesar entidades detectadas
    # Agrupar nombres compuestos (por ejemplo, 'Juan Pérez' debe ser una única entidad)
    temp_names = []
    for entity in entities:
        if entity['entity_group'] == 'PER':  # Nombres de personas
            # Eliminar fragmentos innecesarios como '##v' o tokens mal formateados
            if not entity['word'].startswith('##'):
                temp_names.append(entity['word'])

    # Agrupar los nombres compuestos
    if temp_names:
        combined_names = []
        previous_name = None
        for name in temp_names:
            if previous_name:
                # Si el nombre anterior es un apellido y este uno de los primeros nombres, lo combinamos
                combined_names.append(f"{previous_name} {name}" if len(name.split()) == 1 else previous_name)
                previous_name = name
            else:
                previous_name = name
        names = set(combined_names)

    return {
        "names": sorted(names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [36]:
# Procesar cada CV
for i, cv in enumerate(cvs, start=1):
    info = extract_info_with_ner(cv)
    print(f"CV {i}:")
    print(f"Nombres: {info['names']}")
    print(f"Emails: {info['emails']}")
    print(f"Teléfonos: {info['phones']}")
    print("-" * 40)

CV 1:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com']
Teléfonos: ['+1234567890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+6285777124']
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['+6285287404232']
----------------------------------------
CV 5:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: []
Emails: ['me@junae.id']
Teléfonos: ['+6282331472499']
----------------------------------------
CV 7:
Nombres: []
Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com']
Teléfonos: ['02024444490', '09422071490', '09850671175']
----------------------------------------
CV 8:
Nombres: []
Emails: ['loren@shevitz.org']
Teléfonos: ['606142853', '7736651234']
----------------------------------------
CV 9:
Nombres: []
Emails: []
Teléfonos: 

In [37]:
# Cargar el pipeline de NER
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')

email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text):
    # Detectar entidades con el modelo
    entities = ner(text)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Filtrar y mejorar la detección de nombres
    for entity in entities:
        if entity['entity_group'] == 'PER':  # Nombres de personas
            # Asegurarse de que se mantengan nombres compuestos correctamente
            name = entity['word']
            if name not in names:
                names.add(name)

    # Agrupar nombres en caso de que haya múltiples entidades relacionadas
    combined_names = []
    for name in names:
        # Si el nombre tiene más de una palabra (como 'Juan Pérez'), lo guardamos como un nombre compuesto
        if len(name.split()) > 1:
            combined_names.append(name)

    # Retornar los resultados
    return {
        "names": sorted(combined_names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [38]:
# Procesar cada CV
for i, cv in enumerate(cvs, start=1):
    info = extract_info_with_ner(cv)
    print(f"CV {i}:")
    print(f"Nombres: {info['names']}")
    print(f"Emails: {info['emails']}")
    print(f"Teléfonos: {info['phones']}")
    print("-" * 40)

CV 1:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com']
Teléfonos: ['+1234567890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+6285777124']
----------------------------------------
CV 4:
Nombres: []
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['+6285287404232']
----------------------------------------
CV 5:
Nombres: []
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: ['lam Junaedi']
Emails: ['me@junae.id']
Teléfonos: ['+6282331472499']
----------------------------------------
CV 7:
Nombres: []
Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com']
Teléfonos: ['02024444490', '09422071490', '09850671175']
----------------------------------------
CV 8:
Nombres: []
Emails: ['loren@shevitz.org']
Teléfonos: ['606142853', '7736651234']
----------------------------------------
CV 9:
Nombres: []
Emails: [

### Modelo flair

In [39]:
# Cargar el modelo de Flair para NER
tagger = SequenceTagger.load("flair/ner-english")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_flair(text):
    # Crear una Sentence en Flair
    sentence = Sentence(text)

    # Realizar la anotación NER con Flair
    tagger.predict(sentence)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Filtrar y mejorar la detección de nombres
    for entity in sentence.get_spans('ner'):
        if entity.get_label().value == 'PER':  # Nombres de personas
            names.add(entity.text.strip())

    # Retornar los resultados
    return {
        "names": sorted(names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

2024-12-04 23:12:05,096 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [40]:
# Iterar sobre los CVs y aplicar la función para extraer la información
for idx, cv_text in enumerate(cvs, 1):
    extracted_info = extract_info_with_flair(cv_text)
    print(f"CV {idx}: Nombres: {extracted_info['names']}, Emails: {extracted_info['emails']}, Teléfonos: {extracted_info['phones']}")

CV 1: Nombres: ['Michael  Smith'], Emails: [], Teléfonos: []
CV 2: Nombres: [], Emails: ['hello@reallygreatsite.com'], Teléfonos: ['+1234567890']
CV 3: Nombres: [], Emails: ['dhikayudano@gmail.com'], Teléfonos: ['+6285777124']
CV 4: Nombres: ['Dyah Hediyati', 'Jawa Timur', 'Melakukan  pekerjaan  administratif  perusahaan', 'Menganalisa  dan mengolah', 'Petik Wong Gaptek'], Emails: ['dyahhediyati@gmail.com'], Teléfonos: ['+6285287404232']
CV 5: Nombres: ['Alice Clark Al'], Emails: [], Teléfonos: []
CV 6: Nombres: ['Junaedi'], Emails: ['me@junae.id'], Teléfonos: ['+6282331472499']
CV 7: Nombres: ['DR.SANTOSH  KAKADE'], Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com'], Teléfonos: ['02024444490', '09422071490', '09850671175']
CV 8: Nombres: [], Emails: ['loren@shevitz.org'], Teléfonos: ['606142853', '7736651234']
CV 9: Nombres: [], Emails: [], Teléfonos: ['085157115062']


In [41]:
# Cargar el modelo de Flair para NER
tagger = SequenceTagger.load("flair/ner-english")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_flair(text):
    # Crear una Sentence en Flair
    sentence = Sentence(text)

    # Realizar la anotación NER con Flair
    tagger.predict(sentence)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 8 dígitos
            phones.add(cleaned_phone)

    # Extraer y almacenar nombres solo si son etiquetados como 'PER'
    for entity in sentence.get_spans('ner'):
        if entity.get_label().value == 'PER':  # Nombres de personas
            name = entity.text.strip()
            # Verificar que el nombre no esté vacío y que sea relevante
            if name:
                names.add(name)

    # Retornar los resultados
    return {
        "names": sorted(names),
        "emails": sorted(emails),
        "phones": sorted(phones)
    }

2024-12-04 23:12:24,379 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [42]:
# Iterar sobre los CVs y aplicar la función para extraer la información
for idx, cv_text in enumerate(cvs, 1):
    extracted_info = extract_info_with_flair(cv_text)
    print(f"CV {idx}: Nombres: {extracted_info['names']}, Emails: {extracted_info['emails']}, Teléfonos: {extracted_info['phones']}")

CV 1: Nombres: ['Michael  Smith'], Emails: [], Teléfonos: []
CV 2: Nombres: [], Emails: ['hello@reallygreatsite.com'], Teléfonos: ['+1234567890']
CV 3: Nombres: [], Emails: ['dhikayudano@gmail.com'], Teléfonos: ['+6285777124']
CV 4: Nombres: ['Dyah Hediyati', 'Jawa Timur', 'Melakukan  pekerjaan  administratif  perusahaan', 'Menganalisa  dan mengolah', 'Petik Wong Gaptek'], Emails: ['dyahhediyati@gmail.com'], Teléfonos: ['+6285287404232']
CV 5: Nombres: ['Alice Clark Al'], Emails: [], Teléfonos: []
CV 6: Nombres: ['Junaedi'], Emails: ['me@junae.id'], Teléfonos: ['+6282331472499']
CV 7: Nombres: ['DR.SANTOSH  KAKADE'], Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com'], Teléfonos: ['02024444490', '09422071490', '09850671175']
CV 8: Nombres: [], Emails: ['loren@shevitz.org'], Teléfonos: ['606142853', '7736651234']
CV 9: Nombres: [], Emails: [], Teléfonos: ['085157115062']


Modelo combinado Bert y Flair

In [43]:
# Cargar el pipeline de NER para BERT
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Cargar el modelo de Flair para NER
tagger = SequenceTagger.load("flair/ner-english")

# Expresión regular ajustada para teléfonos
phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text):
    # Detectar entidades con el modelo BERT
    entities = ner(text)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Filtrar y mejorar la detección de nombres
    for entity in entities:
        if entity['entity_group'] == 'PER':  # Nombres de personas
            names.add(entity['word'])

    # Retornar los resultados de BERT
    return {
        "names": names,
        "emails": emails,
        "phones": phones
    }

def extract_info_with_flair(text):
    # Crear una Sentence en Flair
    sentence = Sentence(text)

    # Realizar la anotación NER con Flair
    tagger.predict(sentence)

    # Inicializar estructuras de datos
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)

    # Limpiar y filtrar teléfonos
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)  # Mantener solo dígitos y +
        if len(cleaned_phone) >= 8:  # Teléfonos de al menos 10 dígitos
            phones.add(cleaned_phone)

    # Filtrar y mejorar la detección de nombres
    for entity in sentence.get_spans('ner'):
        if entity.get_label().value == 'PER':  # Nombres de personas
            names.add(entity.text.strip())

    # Retornar los resultados de Flair
    return {
        "names": names,
        "emails": emails,
        "phones": phones
    }

# Función para combinar los resultados sin duplicados
def combine_results(results_ner, results_flair):
    # Combinamos los resultados de nombres, correos electrónicos y teléfonos
    combined_names = results_ner["names"].union(results_flair["names"])
    combined_emails = results_ner["emails"].union(results_flair["emails"])
    combined_phones = results_ner["phones"].union(results_flair["phones"])

    return {
        "names": sorted(combined_names),
        "emails": sorted(combined_emails),
        "phones": sorted(combined_phones)
    }

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


2024-12-04 23:12:43,464 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [44]:
for idx, cv_text in enumerate(cvs, 1):
    # Extraer la información con ambos modelos
    info_ner = extract_info_with_ner(cv_text)
    info_flair = extract_info_with_flair(cv_text)

    # Combinar los resultados sin duplicados
    combined_info = combine_results(info_ner, info_flair)

    # Mostrar resultados
    print(f"CV {idx}:")
    print(f"Nombres: {combined_info['names']}")
    print(f"Emails: {combined_info['emails']}")
    print(f"Teléfonos: {combined_info['phones']}")
    print("-" * 40)

CV 1:
Nombres: ['Michael', 'Michael  Smith']
Emails: []
Teléfonos: []
----------------------------------------
CV 2:
Nombres: []
Emails: ['hello@reallygreatsite.com']
Teléfonos: ['+1234567890']
----------------------------------------
CV 3:
Nombres: []
Emails: ['dhikayudano@gmail.com']
Teléfonos: ['+6285777124']
----------------------------------------
CV 4:
Nombres: ['Dyah Hediyati', 'Jawa Timur', 'Melakukan  pekerjaan  administratif  perusahaan', 'Menganalisa  dan mengolah', 'Petik Wong Gaptek']
Emails: ['dyahhediyati@gmail.com']
Teléfonos: ['+6285287404232']
----------------------------------------
CV 5:
Nombres: ['Alice Clark Al']
Emails: []
Teléfonos: []
----------------------------------------
CV 6:
Nombres: ['Junaedi', 'lam Junaedi']
Emails: ['me@junae.id']
Teléfonos: ['+6282331472499']
----------------------------------------
CV 7:
Nombres: ['##v', 'DR.SANTOSH  KAKADE']
Emails: ['drsantoshkakade@ciilm.com', 'drsantoshkakade@gmail.com']
Teléfonos: ['02024444490', '09422071490', 

##Pipeline


In [ ]:
!pip install sentence-transformers
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install ocrmypdf PyPDF2
!apt-get install -y ghostscript
!pip install flair
!pip install transformers

In [46]:
import re
from google.colab import files
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import pipeline
from flair.data import Sentence
from flair.models import SequenceTagger
import ocrmypdf
import PyPDF2


In [47]:
carpeta_pdf = './CVs'
carpeta_txt = './Texto_2'

os.makedirs(carpeta_pdf, exist_ok=True)
os.makedirs(carpeta_txt, exist_ok=True)

uploaded = files.upload()

for archivo in uploaded.keys():
    os.rename(archivo, os.path.join(carpeta_pdf, archivo))
carpeta_pdf = '/content/CVs'
carpeta_txt = '/content/Texto_2'
os.makedirs(carpeta_pdf, exist_ok=True)
os.makedirs(carpeta_txt, exist_ok=True)

for archivo in os.listdir(carpeta_pdf):
    if archivo.endswith('.pdf'):
        archivo_pdf = os.path.join(carpeta_pdf, archivo)
        archivo_pdf_con_ocr = os.path.join(carpeta_txt, f'{os.path.splitext(archivo)[0]}_ocr.pdf')
        ocrmypdf.ocr(archivo_pdf, archivo_pdf_con_ocr, force_ocr=True)
        texto_extraido = ""
        with open(archivo_pdf_con_ocr, 'rb') as pdf_con_ocr:
            pdf_reader = PyPDF2.PdfReader(pdf_con_ocr)
            for pagina in pdf_reader.pages:
                texto_extraido += pagina.extract_text()
        archivo_salida = os.path.join(carpeta_txt, f'{os.path.splitext(archivo)[0]}.txt')
        with open(archivo_salida, 'w', encoding='utf-8') as f:
            f.write(texto_extraido)


Saving 1724342646_CV1.pdf to 1724342646_CV1.pdf
Saving 1724342661_CV2.pdf to 1724342661_CV2.pdf
Saving 1724342673_CV3.pdf to 1724342673_CV3.pdf
Saving 1724342683_CV4.pdf to 1724342683_CV4.pdf
Saving 1724342694_CV5.pdf to 1724342694_CV5.pdf
Saving 1724342704_CV6.pdf to 1724342704_CV6.pdf
Saving 1724342714_CV7.pdf to 1724342714_CV7.pdf
Saving 1724342728_CV8.pdf to 1724342728_CV8.pdf
Saving 1724342738_CV9.pdf to 1724342738_CV9.pdf


Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [48]:
modelo = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')# se generan los embeddings
tagger = SequenceTagger.load("flair/ner-english")
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

frases_ia = [
    "Machine Learning", "Deep Learning", "Artificial Intelligence", "Neural Networks",
    "Computer Vision", "Natural Language Processing", "Reinforcement Learning",
    "Supervised Learning", "Unsupervised Learning", "Semi-Supervised Learning",
    "Transfer Learning", "Explainable AI", "Edge Computing", "Generative AI",
    "Big Data", "Data Mining", "Data Science", "Feature Engineering", "Model Optimization",
    "Predictive Analytics", "Clustering Algorithms", "Classification Algorithms",
    "Regression Algorithms", "Decision Trees", "Support Vector Machines",
    "Convolutional Neural Networks", "Recurrent Neural Networks", "Transformers",
    "Gradient Boosting", "Hyperparameter Tuning", "AutoML", "AI Ethics", "Fairness in AI",
    "Bias Detection", "GPT", "BERT", "LLM (Large Language Models)", "NLP", "OCR",
    "Speech Recognition", "Text-to-Speech", "Speech-to-Text", "GANs (Generative Adversarial Networks)",
    "RL (Reinforcement Learning)", "AI for Healthcare", "AI for Finance", "AI for Education",
    "AI for Industry 4.0", "Autonomous Systems", "AI Deployment", "AI Scalability", "AI Frameworks",
    "AI Research", "Inteligencia Artificial", "Aprendizaje Automático", "Aprendizaje Supervisado",
    "Aprendizaje No Supervisado", "Aprendizaje Semi-Supervisado", "Aprendizaje por Refuerzo",
    "Aprendizaje Profundo", "Redes Neuronales", "Visión por Computadora",
    "Procesamiento de Lenguaje Natural", "Analítica Predictiva", "Minería de Datos", "Ciencia de Datos",
    "Ingeniería de Características", "Optimización de Modelos", "Análisis Predictivo", "Clústeres",
    "Algoritmos de Clasificación", "Algoritmos de Regresión", "Árboles de Decisión",
    "Máquinas de Soporte Vectorial", "Redes Neuronales Convolucionales",
    "Redes Neuronales Recurrentes", "Transformadores", "Ajuste de Hiperparámetros",
    "Modelos Explicables", "Modelos de Lenguaje Grande", "Toma de Decisiones Autónomas",
    "Inteligencia Artificial Responsable", "Ética en IA", "Detección de Sesgos", "Reconocimiento de Voz",
    "Texto a Voz", "Voz a Texto", "Generación de Imágenes", "Sistemas Autónomos", "IA en la Salud",
    "IA en Finanzas", "IA en la Industria", "IA en la Educación", "Computación en el Borde",
    "Datos Masivos", "Automatización de Procesos", "Investigación en IA"
]

embeddings_frases_ia = modelo.encode(frases_ia)

carpeta_txt = './Texto_2'
cvs = []
for archivo in os.listdir(carpeta_txt):
    if archivo.endswith('.txt'):
        archivo_ruta = os.path.join(carpeta_txt, archivo)
        with open(archivo_ruta, 'r', encoding='utf-8') as f:
            contenido = f.read()
            cvs.append(contenido)
embeddings_cvs = modelo.encode(cvs)
similitudes = cosine_similarity(embeddings_cvs, embeddings_frases_ia)

def normalizar_texto(texto):
    texto_normalizado = re.sub(r'[^\w\s]', '', texto.lower())
    texto_normalizado = re.sub(r'\s+', ' ', texto_normalizado).strip()
    return texto_normalizado

def calcular_coincidencias(cv, frases_ia):
    cv_normalizado = normalizar_texto(cv)
    frases_encontradas = []
    for frase in frases_ia:
        frase_normalizada = normalizar_texto(frase)
        if frase_normalizada in cv_normalizado:
            frases_encontradas.append(frase)
    return len(frases_encontradas), frases_encontradas


phone_regex = re.compile(r'\+?\d{1,3}?[ -]?\(?\d{2,4}\)?[ -]?\d{2,4}[ -]?\d{4,6}')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

def extract_info_with_ner(text): # para extraer información
    entities = ner(text)
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)
        if len(cleaned_phone) >= 8:
            phones.add(cleaned_phone)
    for entity in entities:
        if entity['entity_group'] == 'PER':
            names.add(entity['word'])
    return {
        "names": names,
        "emails": emails,
        "phones": phones
    }

def extract_info_with_flair(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    names = set()
    emails = set(email_regex.findall(text))
    raw_phones = phone_regex.findall(text)
    phones = set()
    for phone in raw_phones:
        cleaned_phone = re.sub(r'[^\d+]', '', phone)
        if len(cleaned_phone) >= 8:
            phones.add(cleaned_phone)
    for entity in sentence.get_spans('ner'):
        if entity.get_label().value == 'PER':
            names.add(entity.text.strip())
    return {
        "names": names,
        "emails": emails,
        "phones": phones
    }

def combine_results(results_ner, results_flair):
    combined_names = results_ner["names"].union(results_flair["names"])
    combined_emails = results_ner["emails"].union(results_flair["emails"])
    combined_phones = results_ner["phones"].union(results_flair["phones"])
    return (sorted(combined_names), sorted(combined_emails), sorted(combined_phones))

def extract_experience(text):
    experience_patterns = [
        r'(\d+(\.\d+)?)\s*(year|años?|yrs?|año)(s)?\s*(experience)?',
        r'(\d+(\.\d+)?)\s*(or more|o más)\s*(years?|años?)',
        r'(over|más de)\s*(\d+)\s*(years?|años?)',
        r'(\d+)\s*(year|años?)\s*(experience|de experiencia)?'
    ]

    experience_years = []

    for pattern in experience_patterns:
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        for match in matches:
            try:
                years = float(match[0])
                if years not in experience_years:
                    experience_years.append(years)
            except ValueError:
                continue
    return experience_years


def calcular_puntaje(cvs:list, frases_ia:list, embeddings_cvs:list, embeddings_frases:list, w1:float=0.8, w2:float=0.2, factor_coincidencias:int=16, factor_similitud:int=9) -> list:
    puntajes_ajustados = []
    if w1 + w2 != 1:
        raise ValueError("La suma de los pesos w1 y w2 debe ser 1.")
    for cv, embedding_cv in zip(cvs, embeddings_cvs):
        coincidencias = 0
        similitudes_semanticas = []
        for frase in frases_ia:
            if frase.lower() in cv.lower():
                coincidencias += 1
        for embedding_frase in embeddings_frases_ia:
            similitud = np.dot(embedding_cv, embedding_frase) / (np.linalg.norm(embedding_cv) * np.linalg.norm(embedding_frase))
            similitudes_semanticas.append(similitud)
        similitud_promedio = np.mean(similitudes_semanticas) if similitudes_semanticas else 0
        coincidencias_ajustadas = coincidencias * factor_coincidencias
        similitud_ajustada = similitud_promedio * factor_similitud
        puntaje_ajustado = (w1 * coincidencias_ajustadas) + (w2 * similitud_ajustada)
        min_local = 0
        max_local = 30
        puntaje_escalado = 1 + 9 * (puntaje_ajustado - min_local) / (max_local - min_local)
        puntajes_ajustados.append(puntaje_escalado)

    return puntajes_ajustados

resultados_finales = []
resultados_finales_esperados = []

for idx, cv_text in enumerate(cvs, 1):
    names, emails, phones = combine_results(extract_info_with_ner(cv_text), extract_info_with_ner(cv_text))
    coincidencias, frases_coincidentes = calcular_coincidencias(cv_text, frases_ia)
    tiene_formacion_ia = 'S' if coincidencias > 0 else 'N'
    puntaje = round(calcular_puntaje(cvs, frases_ia, embeddings_cvs, embeddings_frases_ia)[idx-1],2)
    experience_years = extract_experience(cv_text)

    if experience_years:
        experiencia = str(sum(experience_years))
    else:
        experiencia = 'No encontrado'

    resultado = {
        "Nombre": names if names else 'No encontrado',
        "Teléfono": phones if phones else 'No encontrado',
        "Email": emails if emails else 'No encontrado',
        "Años de experiencia": experiencia,
        "¿Tiene formación en IA? (S/N)": tiene_formacion_ia,
        "Puntaje": puntaje
    }

    diccionario_esperado = {
        "Nombre": names if names else None,
        "Teléfono": phones if phones else None,
        "Email": emails if emails else None,
        "Años de experiencia": experiencia if experience_years else None,
        "¿Tiene formación en IA? (S/N)": tiene_formacion_ia,
        "Puntaje": 0 if not names or not emails or not phones or not experiencia else puntaje
    }

    resultados_finales.append(resultado)
    resultados_finales_esperados.append(diccionario_esperado)

print('\n\nResultados reales:\n\n')
for resultado in resultados_finales:
    print(resultado)
    print('\n\n')
print('\n'*5)
print('\n\nResultados esperados:\n\n')
for resultado in resultados_finales_esperados:
    print(resultado)
    print('\n\n')


2024-12-04 23:16:57,747 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.




Resultados reales:


{'Nombre': ['Michael'], 'Teléfono': 'No encontrado', 'Email': 'No encontrado', 'Años de experiencia': '1.0', '¿Tiene formación en IA? (S/N)': 'S', 'Puntaje': 4.9}



{'Nombre': 'No encontrado', 'Teléfono': ['+1234567890'], 'Email': ['hello@reallygreatsite.com'], 'Años de experiencia': '10.0', '¿Tiene formación en IA? (S/N)': 'N', 'Puntaje': 1.01}



{'Nombre': 'No encontrado', 'Teléfono': ['+6285777124'], 'Email': ['dhikayudano@gmail.com'], 'Años de experiencia': 'No encontrado', '¿Tiene formación en IA? (S/N)': 'N', 'Puntaje': 1.02}



{'Nombre': 'No encontrado', 'Teléfono': ['+6285287404232'], 'Email': ['dyahhediyati@gmail.com'], 'Años de experiencia': 'No encontrado', '¿Tiene formación en IA? (S/N)': 'N', 'Puntaje': 1.11}



{'Nombre': 'No encontrado', 'Teléfono': 'No encontrado', 'Email': 'No encontrado', 'Años de experiencia': 'No encontrado', '¿Tiene formación en IA? (S/N)': 'S', 'Puntaje': 4.9}



{'Nombre': ['lam Junaedi'], 'Teléfono': ['+6282331472499'],